# Packages

In [1]:
### Import packages ###
import itertools
import numpy as np
import pandas as pd
import sys
sys.path.append('..')
from utils.Auxiliary import CreateParameterVectorFunction
from utils.Auxiliary import FilterJobNames

---

# Create Parameter Vector

In [2]:
# Done: Iris, MONK1, MONK3, Haberman
# Running: Bar7 (need one more)
# Running: FICO, COMPAS 

In [3]:
# Data: Iris  MONK1  MONK3  Bar7 (10)  COMPAS (50) | BankNote (10)  BreastCancer (5)  CarEvaluation (10)  FICO (50)  Haberman
Data = "FICO"
ParameterVector = CreateParameterVectorFunction(Data = Data,
                                  Seed = range(0,50),                     # range(0,50)
                                  RashomonThreshold = 0.5,        # For TreeFarms
                                  DiversityWeight = 0,          # For BatchQBC
                                  DensityWeight = 0,            # For BatchQBC
                                  BatchSize = 1,                           # 1 5 or 10
                                  Partition = "short",                                # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                                  Time = "11:59:00",                                     # [00:59:00, 11:59:00, 6-23:59:00]
                                  Memory = "30000M",                                   # [100M, 30000M, 100000M]
                                  IncludePL_RF=True,       # Passive Learning with RandomForestClassifierPredictor
                                  IncludePL_GPC=True,      # Passive Learning with GaussianProcessClassifierPredictor
                                  IncludePL_BNN=True,      # Passive Learning with BayesianNeuralNetworkPredictor
                                  IncludeBALD_BNN=True,    # BALD with BayesianNeuralNetworkPredictor
                                  IncludeBALD_GPC=True,    # BALD with GaussianProcessClassifierPredictor
                                  IncludeQBC_TreeFarms_Unique=False, # BatchQBC with TreeFarmsPredictor (UniqueErrorsInput=1)
                                  IncludeQBC_TreeFarms_Duplicate=False, # BatchQBC with TreeFarmsPredictor (UniqueErrorsInput=0)
                                  IncludeQBC_RF=True,      # BatchQBC with RandomForestClassifierPredictor
                                  IncludeLFR_TreeFarms_Unique=True, #BatchQBC with LFRPredictor (UniqueErrorsInput=1)
                                  IncludeLFR_TreeFarms_Duplicate=True #BatchQBC with LFRPredictor (UniqueErrorsInput=0)
                                  )

# Evaluate

In [4]:
ParameterVector["JobName"]

0                                  0FI_RF_PL_B1
1               0FI_GPC_PL_B1_KTRBF_KLS1_KNU1.5
2                       0FI_Dlfr_A5_DW0_DEW0_B1
3                        0FI_RF_QBC_DW0_DEW0_B1
4         0FI_GPC_BALD_B1_KTRBF_KLS1_KNU1.5_K20
                         ...                   
395    49FI_BNN_PL_B1_HS50_DR2_E100_LR001_BST32
396            49FI_GPC_PL_B1_KTRBF_KLS1_KNU1.5
397                               49FI_RF_PL_B1
398                    49FI_Ulfr_A5_DW0_DEW0_B1
399                    49FI_Dlfr_A5_DW0_DEW0_B1
Name: JobName, Length: 400, dtype: object

In [5]:
ParameterVector["Output"]

0      FICO/RandomForestClassifierPredictor/Raw/0FI_R...
1      FICO/GaussianProcessClassifierPredictor/Raw/0F...
2      FICO/LFRPredictor/Raw/0FI_Dlfr_A5_DW0_DEW0_B1.pkl
3      FICO/RandomForestClassifierPredictor/Raw/0FI_R...
4      FICO/GaussianProcessClassifierPredictor/Raw/0F...
                             ...                        
395    FICO/BayesianNeuralNetworkPredictor/Raw/49FI_B...
396    FICO/GaussianProcessClassifierPredictor/Raw/49...
397    FICO/RandomForestClassifierPredictor/Raw/49FI_...
398    FICO/LFRPredictor/Raw/49FI_Ulfr_A5_DW0_DEW0_B1...
399    FICO/LFRPredictor/Raw/49FI_Dlfr_A5_DW0_DEW0_B1...
Name: Output, Length: 400, dtype: object

# Only run these ones

In [6]:
# # ### Simulations that failed ###
# FilterText = ['26B7_UA02']
# ParameterVector = FilterJobNames(ParameterVector,FilterText)
# ParameterVector.index = range(0,len(ParameterVector))
# ParameterVector

---

# Save Parameter Vector

In [7]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)